In [1]:
import tensorflow as tf
import pickle, pandas as pd, re, numpy as np, ast, warnings

from joblib import Parallel, delayed

import time
from copy import deepcopy
from collections import defaultdict, OrderedDict
from itertools import chain, starmap
from itertools import product
import unicodedata
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score, precision_score, accuracy_score, log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline

from TurkishStemmer import TurkishStemmer
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
import gensim
from textblob import TextBlob

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
    
reset_graph()

In [3]:
df_full = pd.read_csv("../datasets/movie_data.csv")
df_full.head()

,Language,Movie_ID,Review,Score
0,en,-800777728,i love science fiction and i hate superheroes ...,9
1,en,-800777728,the movie is absolutely incredible all the per...,10
2,en,-1018312192,in a cinematic era dominated by reboots and mi...,8
3,en,-1018312192,movie review on rise of the planet of the apes...,4
4,en,-1018312192,during experiments to find a cure for alzheime...,7


In [4]:
df_full.groupby("Score").count()

,Language,Movie_ID,Review
Score,,,
1,29,29,29
2,21,21,21
3,14,14,14
4,23,23,23
5,83,83,83
6,43,43,43
7,71,71,71
8,207,207,207
9,175,175,175


In [5]:
en_vects = gensim.models.KeyedVectors.load_word2vec_format(r"../../NLP_data/GoogleNews-vectors-negative300.bin", binary=True)

In [6]:
tr_vects = gensim.models.KeyedVectors.load_word2vec_format(r"../../NLP_data/wiki.tr/wiki.tr.vec", binary=False)

In [7]:
turkish_stemmer = TurkishStemmer()
def clean(text, language="en", stem=True):
    global turkish_stemmer
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').lower().decode("ascii")
    
    if language == "tr":
        if stem:
            text= ' '.join([turkish_stemmer.stem(w) for w in text.split()])
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r'[0-9]', '#', text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ", text)
    text = re.sub(r"\+", " ", text)
    text = re.sub(r"\-", " ", text)
    text = re.sub(r"\=", " ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r":", " ", text)
    text = re.sub(r"e(\s)?-(\s)?mail", "email", text)

    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    return TextBlob(text)


In [8]:
VECTOR_SIZE = 300
def vectorize(text, language):
    global VECTOR_SIZE            
    blob = clean(text, language)
    vector = np.zeros(VECTOR_SIZE)
    if len(blob.words) < 1:
        return None

    for word in blob.words:
        try:
            if language == "en":
                vector += globals()["en_vects"][word]
            else:
                vector += globals()["tr_vects"][word]
        except KeyError as e:
#             warnings.warn(str(e))
            continue
    vector /= max(len(blob.words),1)
    return vector

In [9]:
def getvec(x):
    lang, rev = x.split(":::::")
    return vectorize(rev, lang)

In [10]:
# LMSR
def preprocess_data(df, language_column="Language", review_column="Review"):
    LMSR_df = df.copy()
    LMSR_df["lang_rev"] = LMSR_df[[language_column, review_column]].apply(lambda x: x[0]+":::::"+x[1], axis=1)
    LMSR_df["rev_vec"] = LMSR_df["lang_rev"].apply(lambda x:getvec(x))
    LMSR_df.drop(["lang_rev", "Review"], axis=1, inplace=True)
    return LMSR_df

In [11]:
def distance_accuracy(y_true, y_predict):
    res = 0
    for i in range(len(y_true)):
        res += abs(y_true[i]-y_predict[i])
    return 1-res/(len(y_true)*len(set(y_true)))

In [12]:
def get_XYy(LMSR):
    X = np.zeros((len(LMSR), VECTOR_SIZE))
    Y = np.zeros((len(LMSR), VECTOR_SIZE))
    y = np.zeros((len(LMSR)))
    i = 0
    for rev in LMSR.iterrows():
        score = rev[1][2]
        rev_vec = rev[1][3]
        score_vec = rev[1][4]

        X[i] = rev_vec
        Y[i] = score_vec
        y[i] = score

        i += 1
    return X, Y, y

In [13]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [14]:
def sigmoid(x, derive=False):
    if derive:
        return x*(1-x)
    return 1/(1+np.exp(-x))

In [15]:
def get_data_dict(df, get_L2and3=False):
    data_dict = dict() #{language:{score: {movie_id: [rev1, rev2, ..., revn]}}}
    L1 = dict()  # {(languge, score, movie_id): list of reviews with the same score with the same language}
    L2 = dict()  # {(language, score): None}
    L3 = dict()  # {score: None}
    for _, row in df.iterrows():
        lang = row["Language"]
        movie_id = row["Movie_ID"]
        score = row["Score"]
        review = row["rev_vec"]

        data_dict.setdefault(lang, {})
        data_dict[lang].setdefault(score, {})
        data_dict[lang][score].setdefault(movie_id, [])
        data_dict[lang][score][movie_id].append(review)
        
        L1.setdefault((lang, score, movie_id), list())
        L1[(lang, score, movie_id)].append(review)
        if get_L2and3:    
            L2[(lang, score)] = None
            L3[score] = None
    if get_L2and3:
        return data_dict, L1, L2, L3
    return data_dict, L1

In [16]:
def get_L2(LSM_R, data_dict):
    L2 = dict()  # {(language, score): list of movies vectors}
    for language in data_dict:
        for score in data_dict[language]:
            for movie_id in data_dict[language][score]:
                L2.setdefault((language, score), list())
                L2[(language, score)].append(LSM_R[(language, score, movie_id)])
    return L2

In [17]:
def get_L3(LS_MR, data_dict):
    L3 = dict()  # {score: vector of merged languages for that score}
    for language in data_dict:
        for score in data_dict[language]:
            L3.setdefault(score, list())
            L3[score].append(LS_MR[(language, score)])
    return L3

In [18]:
def merge(L, W):
    merged = dict()  # {item: vector of merged subitems}
    for i, item in enumerate(sorted(L)):
        for subitem in L[item]:
            merged.setdefault(item, [np.zeros(VECTOR_SIZE),0])
            merged[item][0] += sigmoid(subitem.dot(W[i]))
            merged[item][1] += 1
    for item in merged:
        merged[item] = merged[item][0]/ merged[item][1]
    return merged

In [19]:
def update_weights(L, delta, W, alpha=0.1):
    for i, k in enumerate(sorted(L)):
        for l in L[k]:
            W[i] += l.T.dot(delta[i]) *alpha
    return W

In [20]:
def get_layer_error(delta, W):
    error = 0
    for i in range(len(delta)):
        error += delta[i].dot(W[i].T)
    return error/len(delta)

In [21]:
def get_layer_delta(error, layer, size):
    delta = np.zeros((size, VECTOR_SIZE))
    j = 0
    for i,k in enumerate(sorted(layer)):
        for l in layer[k]:
            delta[j] = error[i]*sigmoid(l, True)
            j += 1
    return delta

## Training

## Full-batch

In [22]:
def get_score_vects(df, iterations=100, alpha=0.1, random_state=42, W1=None, W2=None, W3=None, W4=None):
    LSMR = preprocess_data(df)
    data_dict, L1 = get_data_dict(LSMR)
    y = softmax(list(LSMR.Score))
#     np.random.seed(random_state)
    learning_curve = dict()
    for i in range(iterations+1):
        # forward propagation
        if W1 is None:
            W1 = 2*np.random.random((len(L1), 300, 300))-1

        LSM_R = merge(L1, W1)
        L2 = get_L2(LSM_R, data_dict)
        if W2 is None:
            W2 = 2*np.random.random((len(L2), 300, 300))-1

        LS_MR = merge(L2, W2)
        L3 = get_L3(LS_MR, data_dict)
        if W3 is None:
            W3 = 2*np.random.random((len(L3), 300, 300))-1

        score_vectors_dict = merge(L3, W3)
        l4 = sigmoid(np.array([v for k, v in sorted(score_vectors_dict.items())]))
        if W4 is None:
            W4 = 2*np.random.random((300, len(LSMR)))-1
        
        l5 = softmax(l4.dot(W4))  # predicted scores
        
        # Calculate the error
        l5_error = np.mean(np.dot(np.log(l5), y))
        
        # Back propagation
        l5_delta = l5_error * sigmoid(l5, True)
        W4 += l4.T.dot(l5_delta)*alpha
        
        l4_error = l5_delta.dot(W4.T)
        l4_delta = l4_error * sigmoid(l4, True)
        
        W3 = update_weights(L3, l4_delta, W3, alpha)
        
        l3_error = get_layer_error(l4_delta, W3)
        l3_delta = get_layer_delta(l3_error, L3, len(L2))
        
        W2 = update_weights(L2, l3_delta, W2, alpha)
        
        l2_error = get_layer_error(l3_delta, W2)
        l2_delta = get_layer_delta(l2_error, L2, len(LSMR))
        
        W1 = update_weights(L1, l2_delta, W1, alpha)
        learning_curve[i] = l5_error
        if i%10 == 0:
            print("epoch {}:\t{}".format(i, np.abs(l5_error)))
        if i%100 == 0:
            alpha *= 0.9
    return LSMR, score_vectors_dict, learning_curve

In [23]:
def fit(LSMR, score_vect_dicts,random_state=42, regressor=MLPRegressor(), classifier=MLPClassifier()):
    LSMR["score_vec"] = LSMR["Score"].apply(lambda x: score_vect_dicts[x] if x in score_vect_dicts else np.NaN)
    LSMR.dropna(inplace=True)
    
    X, Y, y = get_XYy(LSMR)
    
    regressor.random_state = random_state
    classifier.random_state = random_state
        
    regressor.fit(X, Y)
    classifier.fit(Y, y)
    return regressor, classifier

In [24]:
def predict(LSMR, score_vect_dicts, regressor, classifier):
    LSMR["score_vec"] = LSMR["Score"].apply(lambda x: score_vect_dicts[x] if x in score_vect_dicts else np.NaN)
    LSMR.dropna(inplace=True)
    
    X, Y, y = get_XYy(LSMR)
    
    preds_score_vecs = regressor.predict(X)
    pred_scores = classifier.predict(preds_score_vecs)
    
    return pred_scores, y

In [60]:
def get_separate_test_indices(df,size=100):
    tr_reviews = df[df.Language=="tr"]
    classes = list(set(tr_reviews["Score"]))
    indices = []
    for s in classes:
        indices += list(np.random.choice(tr_reviews[tr_reviews.Score==s].index, int(size/len(classes))))
    return indices

In [61]:
tr_test_indices = get_separate_test_indices(df_full)

tronly_test_raw = df_full.loc[tr_test_indices]
tronly_test = preprocess_data(tronly_test_raw)
tronly_test[tronly_test.Language=="en"].count()

Language    0
Movie_ID    0
Score       0
rev_vec     0
dtype: int64

In [62]:
tronly_test_raw.groupby("Score").count()

,Language,Movie_ID,Review
Score,,,
1,11,11,11
2,11,11,11
4,11,11,11
5,11,11,11
6,11,11,11
7,11,11,11
8,11,11,11
9,11,11,11
10,11,11,11


In [63]:
df = df_full.drop(tr_test_indices)
df.groupby("Score").count()

,Language,Movie_ID,Review
Score,,,
1,22,22,22
2,13,13,13
3,14,14,14
4,19,19,19
5,72,72,72
6,34,34,34
7,63,63,63
8,197,197,197
9,164,164,164


In [232]:
def eval_models(model, train, test, tronly, ytrain, ytest, ytronly, LL=False, predtrain=False):
    
    _ = time.time()
    model.fit(train, ytrain)
    trat = time.time()-_
    
    _ = time.time()
    predtest= model.predict(test)
    tet = time.time()-_
    s_test = distance_accuracy(ytest, predtest)
    f1_test = f1_score(ytest, predtest, average='weighted')
    
    _ = time.time()
    predtronly = model.predict(tronly)
    trt = time.time()-_
    s_tr = distance_accuracy(ytronly, predtronly)
    f1_tronly = f1_score(ytronly, predtronly, average='weighted')
    
    evals = OrderedDict()

    evals["Test"] = s_test
    evals["Tr. Only"] = s_tr
    evals["Training Time"] = trat
    evals["Testing Time"] = tet
    evals["Tr.Test Time"] = trt
    evals["F1 Test"] = f1_test
    evals["F1 Tr. only"] = f1_tronly
    if LL:
        evals["Test_LL"] = np.NAN
        evals["Tr. Only_LL"] = np.NAN
    if predtrain:
        if LL:
            evals["Train_LL"] = np.NAN
        _ = time.time()
        predicted_train = model.predict(train)
        predtra = time.time()-_
        s_train = distance_accuracy(ytrain, predicted_train)
        f1_train = f1_score(ytrain, predicted_train, average='weighted')
        evals["F1 Train"] = f1_train
        evals["Train"] = s_train
        evals["Pred.Tra. Time"] = predtra
    return evals

In [203]:
def get_total_average(scores_tables):
#     scores_tables: {i_th trial:
#                     {k_th fold:
#                         {'Model': {'Test': 0.8090301003344482,
#                                    'Train': 0.783361064891847,
#                                    'Turkish only': 0.7414285714285714}}}
    avgs = dict()
    for trial in scores_tables:
        for table in scores_tables[trial]:
            for model in scores_tables[trial][table]:
                avgs.setdefault(model, dict())
                for metric, score in scores_tables[trial][table][model].items():
                    avgs[model].setdefault(metric, list())
                    avgs[model][metric].append(score)
    for model in avgs:
        for metric in avgs[model]:
            avgs[model][metric] = np.mean(avgs[model][metric])
    return pd.DataFrame(avgs)

In [204]:
def get_trial_score(trial_scores_tables):
#  trial_scores_tables: {k_th fold:
#                             {'Model': {'Test': 0.8090301003344482,
#                                        'Train': 0.783361064891847,
#                                        'Turkish only': 0.7414285714285714}}}
    avgs = dict()
    for table in trial_scores_tables:
        for model in trial_scores_tables[table]:
            avgs.setdefault(model, dict())
            for metric, score in trial_scores_tables[table][model].items():
                avgs[model].setdefault(metric, list())
                avgs[model][metric].append(score)
    for model in avgs:
        for metric in avgs[model]:
            avgs[model][metric] = np.mean(avgs[model][metric])
    return pd.DataFrame(avgs)

In [205]:
def eval_selectivewaves_regclass(df_, tronly_test_raw, NUM_TRIALS=1, splits=10):  
    df = df_.reset_index(drop=True)
    learning_curves = dict()
    scores_tables = OrderedDict()
    tronly_test = preprocess_data(tronly_test_raw)
    for i in range(NUM_TRIALS):
        print("Trial:\t{}".format(i+1))
        scores_tables[i] = OrderedDict()
        learning_curves[i] = OrderedDict()
        k = 0
        skf = StratifiedKFold(n_splits=splits, random_state=i)
        
        for train_index, test_index in skf.split(df["Review"], df["Language"]):
            print("K:\t{}".format(k+1))
            scores_tables[i][k] = OrderedDict()
            start = time.time()
            LSMR, score_vect_dicts, training_curve = get_score_vects(
                df.loc[train_index], random_state=i, alpha=1e-5, iterations=50)
            regressor, classifier = fit(LSMR, score_vect_dicts, random_state=i)
            trat = time.time()- start

            test_data = preprocess_data(df.loc[test_index])
            _ = time.time()
            preds, true = predict(test_data, score_vect_dicts, regressor, classifier)
            tet = time.time()-_

            _ = time.time()
            preds_train, true_train = predict(preprocess_data(df.loc[train_index]),
                                              score_vect_dicts,
                                              regressor, classifier)
            predtra = time.time()-_

            _ = time.time()
            preds_tr, true_tr = predict(tronly_test, score_vect_dicts, regressor, classifier)
            trt = time.time()-_

            elapsed = time.time()-start

            s = distance_accuracy(true, preds)
            f1_test = f1_score(true, preds, average='weighted')

            s_train = distance_accuracy(true_train, preds_train)
            f1_train = f1_score(true_train, preds_train, average='weighted')

            s_tr = distance_accuracy(true_tr, preds_tr)
            f1_tronly = f1_test = f1_score(true_tr, preds_tr, average='weighted')        


            lr = LogisticRegression(random_state=i)
            mlp = MLPClassifier(random_state=i)
            rf = RandomForestClassifier(random_state=i,n_jobs=-1)
            train_mat = np.array(list(LSMR["rev_vec"]))
            test_mat = np.array(list(test_data["rev_vec"]))
            tronly_mat = np.array(list(tronly_test["rev_vec"]))

            evals = OrderedDict()
            evals["Train"] = s_train
            evals["Test"] = s
            evals["Tr. Only"] = s_tr
            evals["Training Time"] = trat
            evals["Pred.Tra. Time"] = predtra
            evals["Testing Time"] = tet
            evals["Tr.Test Time"] = trt
            evals["F1 Test"] = f1_test
            evals["F1 Train"] = f1_train
            evals["F1 Tr. only"] = f1_tronly
            scores_tables[i][k]["DeepSelect"] = evals
            scores_tables[i][k]["MLP"] = eval_models(
                mlp, train_mat, test_mat, tronly_mat, true_train, true, true_tr)
            scores_tables[i][k]["Logistic Regression"] = eval_models(
                lr, train_mat, test_mat, tronly_mat, true_train, true, true_tr)
            scores_tables[i][k]["RandomForest"] = eval_models(
                rf, train_mat, test_mat, tronly_mat, true_train, true, true_tr)

            print()
            print("K:\t{}".format(k+1))
            print(pd.DataFrame(scores_tables[i][k]))
            print("\nThis fold took:", elapsed, "seconds\n")
            learning_curves[i][k] = training_curve
            k += 1
            print("*"*10+"\n")
        print("Average scores for trial {}".format(i))
        print(get_trial_score(scores_tables[i]))
        print("-"*30)
    print("%%"*20)
    print("Average of {} trials".format(NUM_TRIALS))
    print(get_total_average(scores_tables))
    return scores_tables

In [98]:
scores_tables = eval_selectivewaves_regclass(df, tronly_test_raw)
pickle.dump(scores_tables, open("../results/batch_no_tf_tables.results", "wb"))

Trial:	1
K:	1
epoch 0:	27.087478343729618
epoch 10:	26.57558277245665
epoch 20:	26.046982903887965
epoch 30:	25.591536124258603
epoch 40:	25.205334499737532
epoch 50:	24.836572533010006


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



K:	1
                DeepSelect       MLP  Logistic Regression  RandomForest
F1 Test           0.064553  0.187732             0.199164      0.252325
F1 Tr. only       0.064553  0.089139             0.057121      0.097973
F1 Train          0.389965  0.565241             0.347702      0.989119
Pred.Tra. Time    1.924803  1.613086             0.158243      0.108836
Test              0.799731  0.778226             0.793011      0.803763
Testing Time      0.010773  0.000325             0.000103      0.104990
Tr. Only          0.632997  0.643098             0.643098      0.635241
Tr.Test Time      0.009949  0.000321             0.000098      0.102400
Train             0.848188  0.878502             0.837681      0.996860
Training Time    17.599797  0.001927             0.000732      0.102657

This fold took: 19.793676614761353 seconds

**********

K:	2
epoch 0:	25.19901591461401
epoch 10:	24.782849680988356
epoch 20:	24.55355017161575
epoch 30:	24.404704002828826
epoch 40:	24.27720230523009

/usr/local/lib/python3.5/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



K:	2
                DeepSelect       MLP  Logistic Regression  RandomForest
F1 Test           0.022426  0.233954             0.230137      0.212696
F1 Tr. only       0.022426  0.053324             0.059326      0.035308
F1 Train          0.256932  0.556177             0.339311      0.985555
Pred.Tra. Time    1.769623  1.586268             0.166189      0.108715
Test              0.789130  0.795652             0.798913      0.792391
Testing Time      0.009800  0.011693             0.000104      0.105778
Tr. Only          0.676768  0.620651             0.632997      0.622896
Tr.Test Time      0.013284  0.005169             0.000099      0.105454
Train             0.829312  0.880217             0.841496      0.996984
Training Time    15.942431  0.001550             0.000828      0.103501

This fold took: 17.94920539855957 seconds

**********

K:	3
epoch 0:	22.330001807262242
epoch 10:	22.138615499687234
epoch 20:	22.028359736816785
epoch 30:	21.93013722554206
epoch 40:	21.82146438695881

...

/usr/local/lib/python3.5/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)



K:	8
                DeepSelect       MLP  Logistic Regression  RandomForest
F1 Test           0.049726  0.280187             0.288097      0.280072
F1 Tr. only       0.049726  0.078271             0.061235      0.072251
F1 Train          0.247743  0.555395             0.333347      0.990360
Pred.Tra. Time    1.703305  1.388909             0.154752      0.107956
Test              0.819293  0.828804             0.828804      0.802989
Testing Time      0.009133  0.000485             0.000099      0.105214
Tr. Only          0.578002  0.647587             0.646465      0.615039
Tr.Test Time      0.013395  0.000318             0.000098      0.106031
Train             0.812063  0.879011             0.834258      0.997949
Training Time    14.325004  0.001900             0.000651      0.106191

This fold took: 16.274826049804688 seconds

**********

K:	9
epoch 0:	23.1435304458848
epoch 10:	22.911927464570994
epoch 20:	22.71662407123261
epoch 30:	22.578645247118054
epoch 40:	22.447276366001198

/usr/local/lib/python3.5/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



K:	9
                DeepSelect       MLP  Logistic Regression  RandomForest
F1 Test           0.022676  0.271372             0.253842      0.277655
F1 Tr. only       0.022676  0.054934             0.055009      0.081578
F1 Train          0.283057  0.559113             0.337208      0.993977
Pred.Tra. Time    1.722849  1.386627             0.156608      0.107368
Test              0.796739  0.803261             0.801087      0.792391
Testing Time      0.009026  0.000311             0.000104      0.105310
Tr. Only          0.656566  0.643098             0.620651      0.629630
Tr.Test Time      0.013260  0.000316             0.000100      0.106239
Train             0.835344  0.881423             0.841616      0.998673
Training Time    16.753690  0.001612             0.000794      0.103511

This fold took: 18.688942909240723 seconds

**********

K:	10
epoch 0:	21.08961796524249
epoch 10:	20.866882490994584
epoch 20:	20.69259781072043
epoch 30:	20.515820378903822
epoch 40:	20.3579248995425

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


## Using the full network for prediction
### P.S. this variation supports online (incremental) training

In [99]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [100]:
def get_test(LSMR):
    X = dict()
    y = dict()
    for _, row in LSMR.iterrows():
        score = row["Score"]
        y_ = np.zeros(10)
        y_[score-1] = 1
        y[len(y)] = y_
        X[len(X)] = row["rev_vec"]
    return np.array(list(X.values())), np.array(list(y.values()))

In [216]:
def train_selective(df_train,epochs=10, learning_rate = 0.1, random_state=42, p_every=10):
    classes = sorted(set(df_train["Score"]))
    n_classes = len(classes)
    
    LSMR_train = preprocess_data(df_train)
    np.random.seed(random_state)
    data_dict, L1, L2, L3 = get_data_dict(LSMR_train, get_L2and3=True)
    init_weights = lambda layer, i, o: {k:2*np.random.random((i, o))-1 for k in layer}
    W1 = init_weights(L1, 300, 300)  # (languge, score, movie_id)
    W2 = init_weights(L2, 300, 300)  # (languge, score):
    W3 = init_weights(L3, 300, n_classes)  # score:
    
    reset_graph()
    x = tf.placeholder(tf.float32, [None, 300])
    y = tf.placeholder(tf.float32, [None, n_classes]) # 1-10 => 10 classes

    w1 = tf.placeholder(tf.float32, [300, 300])
    w2 = tf.placeholder(tf.float32, [300, 300])
    w3 = tf.placeholder(tf.float32, [300, n_classes])

    b1 = tf.Variable(tf.zeros([300]))
    b2 = tf.Variable(tf.zeros([300]))
    b3 = tf.Variable(tf.zeros([n_classes]))

    l2 = tf.nn.sigmoid(tf.matmul(x, w1) + b1)
    l3 = tf.nn.sigmoid(tf.matmul(l2, w2) + b2)
    logits = tf.matmul(l3, w3) + b3
    pred = tf.nn.softmax(logits)

#     cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=classes, logits=logits))
#     optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
    
    
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
    training_curve = dict()
    with tf.device('/job:localhost/replica:0/task:0/device:GPU:0'):
        with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
            sess.run(tf.global_variables_initializer())
            for e in range(epochs+1):
                avg_cost = 0.
                for _, row in LSMR_train.iterrows():
                    lang = row["Language"]
                    movie_id = row["Movie_ID"]
                    score = row["Score"]
                    y_ = {i:0 for i in classes}
                    y_[score] = 1
                    y_ = np.atleast_2d([y_[i] for i in classes])
                    x_ = np.atleast_2d(row["rev_vec"])
                    w1_,w2_,w3_,_, c = sess.run([w1, w2, w3, optimizer, cost],
                                             feed_dict={x: x_,
                                                        y: y_,
                                                        w1:W1[(lang, score, movie_id)],
                                                        w2:W2[(lang, score)],
                                                        w3:W3[score]})
                    W1[(lang, score, movie_id)] = w1_
                    W2[(lang, score)] = w2_
                    W3[score] = w3_

                    avg_cost += c
                training_curve[e] = avg_cost
                if e%10==0:
                    learning_rate /= 10
                if e%p_every==0:
                    print("Epoch {}: {}".format(e, avg_cost/len(LSMR_train)))
    print("W1 size",len(W1))
    print("W2 size",len(W2))
    print("W3 size",len(W3))
    return W1, W2, W3, training_curve, classes

In [217]:
def get_max_index(predicted_scores, classes):
    probs = {c:0 for c in classes}
    array = softmax(predicted_scores)
    indx = None
    max_ = float("-inf")
    for i, e in enumerate(array):
        probs[predicted_scores[i]] = e
        if e > max_:
            max_ = e
            indx = i
    return predicted_scores[indx], [probs[k] for k in sorted(classes)]

In [218]:
def predict_selective(df, W1, W2, W3, classes=list(range(1,11))):
    LSMR = preprocess_data(df)
    reset_graph()
    n_classes = len(classes)
    x = tf.placeholder(tf.float32, [None, 300])

    w1 = tf.placeholder(tf.float32, [300, 300])
    w2 = tf.placeholder(tf.float32, [300, 300])
    w3 = tf.placeholder(tf.float32, [300, n_classes])

    b1 = tf.Variable(tf.zeros([300]))
    b2 = tf.Variable(tf.zeros([300]))
    b3 = tf.Variable(tf.zeros([n_classes]))

    l2 = tf.nn.sigmoid(tf.matmul(x, w1) + b1)
    l3 = tf.nn.sigmoid(tf.matmul(l2, w2) + b2)
    pred = tf.nn.softmax(tf.matmul(l3, w3) + b3)

    
    
    
    prediction = tf.argmax(pred, 1)
    preds = np.zeros(len(LSMR))
    probs = [None] * len(LSMR)
    with tf.device('/job:localhost/replica:0/task:0/device:GPU:0'):
        with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
            sess.run(tf.global_variables_initializer())
            j = 0
            for _, row in LSMR.iterrows():
                v = row["rev_vec"]
                predicted_scores = np.zeros(len(W1))
                for i, info in enumerate(W1):
                    language, score, movie_id = info
                    w_1 = W1[(language, score, movie_id)]
                    w_2 = W2[(language, score)]
                    w_3 = W3[score]

                    predicted_scores[i] = prediction.eval({x: np.atleast_2d(v),
                                                           w1:w_1,w2:w_2,w3:w_3})
                predicted_score, probabilities = get_max_index(predicted_scores, classes)
                preds[j] = predicted_score
                probs[j] = probabilities
                j+=1


    return preds, np.array(list(LSMR.Score)), probs

In [226]:
def eval_selectivewaves_nn(df_, tronly_test_raw, NUM_TRIALS=1, splits=10, iterations=150, alpha=0.1, p_every=25):  
    df = df_.reset_index(drop=True)
    learning_curves = OrderedDict()
    scores_tables_nn = OrderedDict()
    tronly_test = preprocess_data(tronly_test_raw)
    for i in range(NUM_TRIALS):
        print("Trial:\t{}".format(i+1))
        learning_curves[i] = OrderedDict()
        k = 0
        skf = StratifiedKFold(n_splits=splits, random_state=i)
        scores_tables_nn[i] = dict()
        for train_index, test_index in skf.split(df["Review"], df["Language"]):
            print("K: \t{}".format(k+1))
            scores_tables_nn[i][k] = OrderedDict()
            start = time.time()
            # approx 3 epochs per second
            LSMR = preprocess_data(df.loc[train_index])
            W1, W2, W3, training_curve, classes = train_selective(
                df.loc[train_index], epochs=iterations, p_every=p_every, learning_rate=alpha)
            _ = time.time()
            trat = _-start
            print("Took: {} for training".format(trat))
#             true_train = np.array(list(LSMR.Score))
            _ = time.time()
            preds_train, true_train, probs = predict_selective(df.loc[train_index], W1, W2, W3, classes=classes)
            ll_train = log_loss(true_train,probs, labels=classes)
            predtra = time.time()-_
            print("Took: {} for predicting {} training instances".format(predtra, len(train_index)))

            test_data = preprocess_data(df.loc[test_index])
            _ = time.time()
            preds, true, probs = predict_selective(df.loc[test_index], W1, W2, W3, classes=classes)
            ll_test = log_loss(true,probs, labels=classes)
            tet = time.time()-_
            print("Took: {} for predicting {} test instances".format(tet, len(test_index)))

            _ = time.time()
            preds_tr, true_tr, probs = predict_selective(tronly_test_raw, W1, W2, W3, classes=classes)
            ll_tr = log_loss(true_tr,probs, labels=classes)
            trt = time.time()-_
            print("Took: {} for predicting {} Turkish test instances".format(trt, len(tronly_test)))

            elapsed = time.time()-start

            s = distance_accuracy(true, preds)
            s_train = distance_accuracy(true_train, preds_train)
            s_tr = distance_accuracy(true_tr, preds_tr)

            f1_test = f1_score(true, preds, average='weighted')
            f1_train = f1_score(true_train, preds_train, average='weighted')
            f1_tronly = f1_score(true_tr, preds_tr, average='weighted')

            mlp = MLPClassifier(random_state=i)
            lr = LogisticRegression(random_state=i)
            rf = RandomForestClassifier(random_state=i,n_jobs=-1)
            train_mat = np.array(list(LSMR["rev_vec"]))
            test_mat = np.array(list(test_data["rev_vec"]))
            tronly_mat = np.array(list(tronly_test["rev_vec"]))

            evals = OrderedDict()
            evals["Train"] = s_train
            evals["Test"] = s
            evals["Tr. Only"] = s_tr
            evals["Training Time"] = trat
            evals["Pred.Tra. Time"] = predtra
            evals["Testing Time"] = tet
            evals["Tr.Test Time"] = trt
            evals["F1 Test"] = f1_test
            evals["F1 Train"] = f1_train
            evals["F1 Tr. only"] = f1_tronly
            evals["Train_LL"] = ll_train
            evals["Test_LL"] = ll_test
            evals["Tr. Only_LL"] = ll_tr
            scores_tables_nn[i][k]["DeepSelect"] = evals

            scores_tables_nn[i][k]["LogisticRegression"] = eval_models(
                lr, train_mat, test_mat, tronly_mat, ytrain=true_train, ytest=true, ytronly=true_tr, LL=True, predtrain=True)
            scores_tables_nn[i][k]["MLP"] = eval_models(
                mlp, train_mat, test_mat, tronly_mat, ytrain=true_train, ytest=true, ytronly=true_tr, LL=True, predtrain=True)
            scores_tables_nn[i][k]["RandomForest"] = eval_models(
                rf, train_mat, test_mat, tronly_mat, ytrain=true_train, ytest=true, ytronly=true_tr, LL=True, predtrain=True)

            print()
            print(pd.DataFrame(scores_tables_nn[i][k]))
            print("took:", elapsed, "seconds\n")
            learning_curves[i][k] = training_curve
            k += 1
            print("*"*10+"\n")
        print("Average scores for trial {}".format(i))
        print(get_trial_score(scores_tables_nn[i]))
        print("-"*30)
    print("%%"*20)
    print("Average of {} trials".format(NUM_TRIALS))
    print(get_total_average(scores_tables_nn))
    return scores_tables_nn

In [233]:
scores_tables_nn = eval_selectivewaves_nn(df, tronly_test_raw)
pickle.dump(scores_tables_nn, open("../results/incremental_tf_tables.results", "wb"))

Trial:	1
K: 	1
Epoch 0: 1.690964459888837
Epoch 25: 0.01062340211847238
Epoch 50: 0.0027972125442851945
Epoch 75: 0.0016094642921393313
Epoch 100: 0.0011625836473166977
Epoch 125: 0.0009180902153019107
Epoch 150: 0.0007621246894794889
W1 size 588
W2 size 19
W3 size 10
Took: 148.44721388816833 for training
Took: 412.4607973098755 for predicting 828 training instances
Took: 45.89663243293762 for predicting 93 test instances
Took: 47.7384614944458 for predicting 99 Turkish test instances


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



                DeepSelect  LogisticRegression       MLP  RandomForest
F1 Test           0.105873            0.199164  0.187732      0.252325
F1 Tr. only       0.022222            0.057121  0.089139      0.097973
F1 Train          0.048908            0.347702  0.565241      0.989119
Pred.Tra. Time  412.460797            0.000673  0.001591      0.105967
Test              0.827957            0.793011  0.778226      0.803763
Test_LL          10.541547                 NaN       NaN           NaN
Testing Time     45.896632            0.000155  0.000544      0.101892
Tr. Only          0.617284            0.643098  0.643098      0.635241
Tr. Only_LL       7.226090                 NaN       NaN           NaN
Tr.Test Time     47.738461            0.000102  0.000324      0.105297
Train             0.836473            0.837681  0.878502      0.996860
Train_LL         12.828454                 NaN       NaN           NaN
Training Time   148.447214            0.159254  1.779774      0.144115
took:

/usr/local/lib/python3.5/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



                DeepSelect  LogisticRegression       MLP  RandomForest
F1 Test           0.046144            0.230137  0.233954      0.212696
F1 Tr. only       0.022222            0.059326  0.053324      0.035308
F1 Train          0.056831            0.339311  0.556177      0.985555
Pred.Tra. Time  406.869361            0.000626  0.001581      0.105656
Test              0.811957            0.798913  0.795652      0.792391
Test_LL          12.465983                 NaN       NaN           NaN
Testing Time     49.102387            0.000160  0.000347      0.102229
Tr. Only          0.617284            0.632997  0.620651      0.622896
Tr. Only_LL       7.204125                 NaN       NaN           NaN
Tr.Test Time     53.307730            0.000106  0.000322      0.105429
Train             0.841616            0.841496  0.880217      0.996984
Train_LL         12.700493                 NaN       NaN           NaN
Training Time   143.312201            0.141908  1.490702      0.111032
took:

...


                DeepSelect  LogisticRegression       MLP  RandomForest
F1 Test           0.070701            0.288097  0.280187      0.280072
F1 Tr. only       0.022222            0.061235  0.078271      0.072251
F1 Train          0.052131            0.333347  0.555395      0.990360
Pred.Tra. Time  465.779839            0.000618  0.001713      0.105831
Test              0.849185            0.828804  0.828804      0.802989
Test_LL          12.396946                 NaN       NaN           NaN
Testing Time     51.143748            0.000171  0.000615      0.101856
Tr. Only          0.617284            0.646465  0.647587      0.615039
Tr. Only_LL       7.124363                 NaN       NaN           NaN
Tr.Test Time     59.678867            0.000103  0.000353      0.104941
Train             0.834741            0.834258  0.879011      0.997949
Train_LL         12.232387                 NaN       NaN           NaN
Training Time   151.083323            0.144717  1.980307      0.112504
took:

/usr/local/lib/python3.5/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



                DeepSelect  LogisticRegression       MLP  RandomForest
F1 Test           0.045713            0.253842  0.271372      0.277655
F1 Tr. only       0.022222            0.055009  0.054934      0.081578
F1 Train          0.054766            0.337208  0.559113      0.993977
Pred.Tra. Time  461.780762            0.000739  0.001635      0.105773
Test              0.820652            0.801087  0.803261      0.792391
Test_LL          10.984515                 NaN       NaN           NaN
Testing Time     54.803491            0.000222  0.000397      0.102280
Tr. Only          0.617284            0.620651  0.643098      0.629630
Tr. Only_LL       7.086265                 NaN       NaN           NaN
Tr.Test Time     53.079778            0.000119  0.000339      0.105478
Train             0.841134            0.841616  0.881423      0.998673
Train_LL         12.256748                 NaN       NaN           NaN
Training Time   147.804289            0.148943  1.494663      0.110694
took:

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)



                DeepSelect  LogisticRegression       MLP  RandomForest
F1 Test           0.040197            0.305875  0.332993      0.327193
F1 Tr. only       0.022222            0.054836  0.077188      0.072809
F1 Train          0.055503            0.332507  0.528994      0.995171
Pred.Tra. Time  465.385186            0.000678  0.001666      0.105781
Test              0.811141            0.794837  0.798913      0.790761
Test_LL          13.651227                 NaN       NaN           NaN
Testing Time     54.720706            0.000145  0.000374      0.101950
Tr. Only          0.617284            0.631874  0.631874      0.640853
Tr. Only_LL       7.125851                 NaN       NaN           NaN
Tr.Test Time     57.025618            0.000121  0.000337      0.105563
Train             0.838118            0.836429  0.874306      0.999397
Train_LL         12.097566                 NaN       NaN           NaN
Training Time   153.208471            0.146303  1.338109      0.134842
took:

# Robustness test

In [108]:
def f1_score_weighted(true, preds):
    return f1_score(true, preds, average='weighted')

In [222]:
DEFAULT_METRICES = {"MAE-accuracy":distance_accuracy, "f1_score w.avg":f1_score_weighted}

def robustness_test(df_full, test, metrics=DEFAULT_METRICES, iterations_nn=150, iterations_regclass=50):
    en_revs = df_full[df_full.Language=="en"]
    tr_revs = df_full[df_full.Language=="tr"]
    robustness = dict()  # {(num of en reviews, num of tr reviews): scores_dict}
    for en_size in range(1,11):
        for tr_size in range(1,11):
            en_train = en_revs.sample(frac=en_size/10.0)
            tr_train = tr_revs.sample(frac=tr_size/10.0)
            start = time.time()
            print("En: {}\tTr: {}".format(len(en_train),len(tr_train)))
            train = pd.concat([en_train, tr_train]).reset_index(drop=True)
            robustness_tables = dict()
            print("Using first variation (Regressor and Classifier with score vectors)")
            LSMR, score_vect_dicts, training_curve = get_score_vects(
                                            train, alpha=1e-5, iterations=iterations_regclass)
            regressor, classifier = fit(LSMR, score_vect_dicts)
            test_vecs = preprocess_data(test)
            preds, true = predict(test_vecs, score_vect_dicts, regressor, classifier)
            robustness_tables["DeepSelect (regclass)"] = dict()
            robustness_tables["DeepSelect"] = dict()

            print("Using second variation (average of outputs produced by each set of weight matrices)")
            W1, W2, W3, training_curve, classes = train_selective(train, epochs=iterations_nn, p_every=25)
            preds_nn, true_nn, probs = predict_selective(test, W1, W2, W3, classes=classes)
            ll = log_loss(true_nn,probs, labels=classes)
            
    #         f1_test_nn = f1_score(true_nn, preds_nn, average='weighted')
        
            for name, metric in metrics.items():
                
                s_regclass = metric(true, preds)
                s_nn = metric(true_nn, preds_nn)
    #         f1_test_regclass = f1_score(true, preds, average='weighted')
                robustness_tables["DeepSelect (regclass)"][name] = s_regclass
                robustness_tables["DeepSelect"][name] = s_nn
            robustness_tables["DeepSelect (regclass)"]["log loss"] = np.NAN
            robustness_tables["DeepSelect"]["log loss"] = ll

            print("Using well-known algorithms: Logistic Regression, RandomForest and MLP")
            lr = LogisticRegression()
            rf = RandomForestClassifier(n_jobs=-1)
            mlp = MLPClassifier()

            train_mat = np.array(list(LSMR["rev_vec"]))
            train_y = np.array(list(LSMR["Score"]))
            test_mat = np.array(list(test_vecs["rev_vec"]))

            for model_name, model in [("Logistic Regression",lr),
                                ("RandomForest", rf),
                                ("MLP", mlp)]:
                model.fit(train_mat, train_y)
                robustness_tables.setdefault(model_name, dict())
                robustness_tables[model_name]["log loss"] = np.NAN
                for metric_name, metric in metrics.items():
                    robustness_tables[model_name][metric_name] = metric(true, model.predict(test_mat))
            robustness[(len(en_train),len(tr_train))] = robustness_tables
            print("Took: {}".format(time.time()-start))
            print("-"*50)
    return robustness

In [112]:
robustness = robustness_test(df, tronly_test_raw)
pickle.dump(robustness, open("../results/robustness.results", "wb"))

En: 50	Tr: 42
Using first variation (Regressor and Classifier with score vectors)
epoch 0:	16.16214025578675
epoch 10:	16.111929013766158
epoch 20:	16.033517423309394
epoch 30:	15.96701407651971
epoch 40:	15.917165631766993
epoch 50:	15.872577424756678
Using second variation (average of outputs produced by each set of weight matrices)
Epoch 0: 7.056428817968876
Epoch 25: 0.022035374260034903
Epoch 50: 0.009983955117743038
Epoch 75: 0.006561515400701113
Epoch 100: 0.0049227987240892225
Epoch 125: 0.0039557820361017584
Epoch 150: 0.0033154513993414857


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Using well-known algorithms: Logistic Regression, RandomForest and MLP


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Took: 29.425021171569824
--------------------------------------------------
En: 50	Tr: 84
Using first variation (Regressor and Classifier with score vectors)
epoch 0:	20.626597460985447
epoch 10:	20.506254884015398
epoch 20:	20.39389621861168
epoch 30:	20.28264915033963
epoch 40:	20.17534565658642
epoch 50:	20.073614064441486
Using second variation (average of outputs produced by each set of weight matrices)
Epoch 0: 5.608208764900468
Epoch 25: 0.05821456690905472
Epoch 50: 0.014138146270786975
Epoch 75: 0.008361474881245504
Epoch 100: 0.006002886781447984
Epoch 125: 0.004709045632275028
Epoch 150: 0.0038875601788908197
Using well-known algorithms: Logistic Regression, RandomForest and MLP


...

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Took: 253.33083987236023
--------------------------------------------------
En: 500	Tr: 421
Using first variation (Regressor and Classifier with score vectors)
epoch 0:	23.265780520111786
epoch 10:	22.966522638733363
epoch 20:	22.750663699379373
epoch 30:	22.555301412770174
epoch 40:	22.385140844972806
epoch 50:	22.228637286119913
Using second variation (average of outputs produced by each set of weight matrices)
Epoch 0: 1.6882746880114516
Epoch 25: 0.008514598274258182
Epoch 50: 0.0024493747350401466
Epoch 75: 0.0013883159054374561
Epoch 100: 0.00098618992089173
Epoch 125: 0.0007700134486349704
Epoch 150: 0.0006340185141618625
Using well-known algorithms: Logistic Regression, RandomForest and MLP


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Took: 245.67016983032227
--------------------------------------------------


/usr/local/lib/python3.5/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


# grouping labels
### label = 1   if review_score > 7
### label = 0   if 7 >= review_score >= 4
### label = -1  if review_score < 4

In [161]:
def group_labels(x):
    if x>7:
        return 1
    elif x>=4:
        return 0
    return -1

In [162]:
df_full_tri = deepcopy(df_full)
df_full_tri["Score"] = df_full_tri.Score.apply(group_labels)
df_full_tri.groupby("Score").count()

,Language,Movie_ID,Review
Score,,,
-1,64,64,64
0,220,220,220
1,716,716,716


In [163]:
tr_tri_test_indices = get_separate_test_indices(df_full_tri)

tronly_test_raw_tri = df_full_tri.loc[tr_tri_test_indices]
tronly_test_tri = preprocess_data(tronly_test_raw_tri)
tronly_test_tri[tronly_test_tri.Language=="en"].count()

Language    0
Movie_ID    0
Score       0
rev_vec     0
dtype: int64

In [164]:
df_tri = df_full_tri.drop(tr_tri_test_indices)
df_tri.groupby("Score").count()

,Language,Movie_ID,Review
Score,,,
-1,47,47,47
0,192,192,192
1,683,683,683


In [117]:
scores_tables_tri = eval_selectivewaves_regclass(df_tri, tronly_test_raw_tri)
pickle.dump(scores_tables_tri, open("../results/batch_no_tf_tables_tri.results", "wb"))

Trial:	1
K:	1
epoch 0:	22.936336931209713
epoch 10:	23.078613645154178
epoch 20:	22.958781129938234
epoch 30:	22.836696634256274
epoch 40:	22.719323573880516
epoch 50:	22.606666754444664


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



K:	1
                DeepSelect       MLP  Logistic Regression  RandomForest
F1 Test           0.166667  0.703290             0.646481      0.685277
F1 Tr. only       0.166667  0.246133             0.187547      0.239488
F1 Train          0.629103  0.831974             0.631934      0.989114
Pred.Tra. Time    1.580238  1.262647             0.054641      0.111169
Test              0.910394  0.910394             0.910394      0.906810
Testing Time      0.008634  0.000292             0.000126      0.105158
Tr. Only          0.666667  0.696970             0.670034      0.676768
Tr.Test Time      0.013511  0.000293             0.000124      0.105922
Train             0.894378  0.942570             0.894779      0.995984
Training Time    10.777735  0.001437             0.000683      0.106041

This fold took: 12.569423913955688 seconds

**********

K:	2
epoch 0:	22.502337542248583
epoch 10:	22.316128548699904
epoch 20:	22.2218707375343
epoch 30:	22.024503701725433
epoch 40:	21.85482742483926

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



K:	2
                DeepSelect       MLP  Logistic Regression  RandomForest
F1 Test           0.166667  0.563848             0.519620      0.581826
F1 Tr. only       0.166667  0.221375             0.162850      0.247387
F1 Train          0.643669  0.762401             0.646507      0.992739
Pred.Tra. Time    1.814959  0.760005             0.053201      0.110842
Test              0.856631  0.867384             0.856631      0.860215
Testing Time      0.008484  0.000292             0.000124      0.105154
Tr. Only          0.666667  0.693603             0.663300      0.683502
Tr.Test Time      0.013696  0.000293             0.000126      0.105128
Train             0.900402  0.921687             0.900803      0.997590
Training Time    10.712398  0.001408             0.000689      0.103191

This fold took: 12.75432276725769 seconds

**********

K:	3
epoch 0:	20.829381375995336
epoch 10:	20.337402636265413
epoch 20:	20.031938075962632
epoch 30:	19.82738020661237
epoch 40:	19.65961185229024

...

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



K:	8
                DeepSelect       MLP  Logistic Regression  RandomForest
F1 Test           0.166667  0.758029             0.762670      0.751643
F1 Tr. only       0.166667  0.225895             0.183150      0.163912
F1 Train          0.616662  0.802598             0.619485      0.996386
Pred.Tra. Time    1.594989  0.945525             0.050484      0.111117
Test              0.938406  0.920290             0.938406      0.923913
Testing Time      0.008300  0.000291             0.000120      0.105270
Tr. Only          0.666667  0.683502             0.666667      0.663300
Tr.Test Time      0.013069  0.000291             0.000123      0.105779
Train             0.891296  0.931408             0.891697      0.998797
Training Time    10.732935  0.001331             0.000684      0.106162

This fold took: 12.556363821029663 seconds

**********

K:	9
epoch 0:	20.662780358138182
epoch 10:	20.484602748713254
epoch 20:	20.364619577050664
epoch 30:	20.2314754826056
epoch 40:	20.11847615332246

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



K:	9
                DeepSelect       MLP  Logistic Regression  RandomForest
F1 Test           0.166667  0.606206             0.565498      0.557423
F1 Tr. only       0.166667  0.280580             0.162850      0.202412
F1 Train          0.637595  0.846404             0.640428      0.993977
Pred.Tra. Time    1.628215  1.192812             0.054575      0.111171
Test              0.865942  0.862319             0.862319      0.851449
Testing Time      0.008475  0.000296             0.000120      0.105234
Tr. Only          0.666667  0.703704             0.663300      0.670034
Tr.Test Time      0.013042  0.000292             0.000125      0.105973
Train             0.899318  0.947453             0.899719      0.997994
Training Time    10.646667  0.001423             0.000647      0.106062

This fold took: 12.478623390197754 seconds

**********

K:	10
epoch 0:	20.95191803805518
epoch 10:	20.588971857585474
epoch 20:	20.363820373135823
epoch 30:	20.17272059332595
epoch 40:	20.0188454875972

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [124]:
scores_tables_nn_tri = eval_selectivewaves_nn(df_tri, tronly_test_raw_tri)
pickle.dump(scores_tables_nn_tri, open("../results/incremental_tf_tables_tri.results", "wb"))

Trial:	1
K: 	1
Epoch 0: 0.28072113833845036
Epoch 25: 0.00038186645831767046
Epoch 50: 0.00020485301775494922
Epoch 75: 0.00014177591513705656
Epoch 100: 0.00010894961932635454
Epoch 125: 8.87160665386721e-05
Epoch 150: 7.495163620116827e-05
Took: 141.4846637248993 for training
Took: 331.2418580055237 for predicting 830 training instances
Took: 37.62711691856384 for predicting 93 test instances
Took: 45.67571544647217 for predicting 99 Turkish test instances


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



                DeepSelect  LogisticRegression       MLP  RandomForest
F1 Test           0.000000            0.646481  0.703290      0.685277
F1 Tr. only       0.000000            0.187547  0.246133      0.239488
F1 Train          0.000000            0.631934  0.831974      0.989114
Pred.Tra. Time  331.241858            0.048902  1.160496      0.126779
Test              0.577061            0.910394  0.910394      0.906810
Test_LL           1.135341                 NaN       NaN           NaN
Testing Time     37.627117            0.000097  0.000329      0.105414
Tr. Only          0.333333            0.670034  0.696970      0.676768
Tr. Only_LL       9.883215                 NaN       NaN           NaN
Tr.Test Time     45.675715            0.000122  0.000301      0.105042
Train             0.561044            0.894779  0.942570      0.995984
Train_LL          2.087202                 NaN       NaN           NaN
Training Time   141.484664            0.000633  0.001352      0.103339
took:

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



                DeepSelect  LogisticRegression       MLP  RandomForest
F1 Test           0.000000            0.519620  0.563848      0.581826
F1 Tr. only       0.000000            0.162850  0.221375      0.247387
F1 Train          0.000000            0.646507  0.762401      0.992739
Pred.Tra. Time  354.174372            0.053123  0.807652      0.126586
Test              0.523297            0.856631  0.867384      0.860215
Test_LL           3.011990                 NaN       NaN           NaN
Testing Time     42.726577            0.000095  0.000314      0.105289
Tr. Only          0.336700            0.663300  0.693603      0.683502
Tr. Only_LL      10.080462                 NaN       NaN           NaN
Tr.Test Time     36.477187            0.000093  0.000294      0.104974
Train             0.567068            0.900803  0.921687      0.997590
Train_LL          1.915997                 NaN       NaN           NaN
Training Time   146.387853            0.000613  0.001368      0.102760
took:

...

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



                DeepSelect  LogisticRegression       MLP  RandomForest
F1 Test           0.300085            0.565498  0.606206      0.557423
F1 Tr. only       0.000000            0.162850  0.280580      0.202412
F1 Train          0.392678            0.640428  0.846404      0.993977
Pred.Tra. Time  357.258294            0.044138  1.122279      0.110796
Test              0.670290            0.862319  0.862319      0.851449
Test_LL           3.324678                 NaN       NaN           NaN
Testing Time     42.013138            0.000095  0.000291      0.104472
Tr. Only          0.336700            0.663300  0.703704      0.670034
Tr. Only_LL      10.179548                 NaN       NaN           NaN
Tr.Test Time     41.037424            0.000099  0.000285      0.105638
Train             0.721219            0.899719  0.947453      0.997994
Train_LL          1.899030                 NaN       NaN           NaN
Training Time   139.331887            0.000536  0.001354      0.102487
took:

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)



                DeepSelect  LogisticRegression       MLP  RandomForest
F1 Test           0.000000            0.687169  0.729318      0.669194
F1 Tr. only       0.000000            0.166667  0.220579      0.254782
F1 Train          0.000000            0.627517  0.815876      0.992759
Pred.Tra. Time  360.673268            0.051123  0.934448      0.110875
Test              0.583333            0.916667  0.920290      0.884058
Test_LL           1.396584                 NaN       NaN           NaN
Testing Time     35.869923            0.000104  0.000319      0.104990
Tr. Only          0.333333            0.666667  0.686869      0.680135
Tr. Only_LL       9.840825                 NaN       NaN           NaN
Tr.Test Time     38.183248            0.000105  0.000315      0.105059
Train             0.560369            0.894103  0.934216      0.997192
Train_LL          2.046992                 NaN       NaN           NaN
Training Time   140.717501            0.000561  0.001580      0.105607
took:

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [234]:
robustness_tri = robustness_test(df_tri, tronly_test_raw_tri)
pickle.dump(robustness_tri, open("../results/robustness_tri.results", "wb"))

En: 50	Tr: 42
Using first variation (Regressor and Classifier with score vectors)
epoch 0:	16.132868979886112
epoch 10:	16.03108636985438
epoch 20:	15.834248752891959
epoch 30:	15.669447353683355
epoch 40:	15.527291443100244
epoch 50:	15.415641292313476
Using second variation (average of outputs produced by each set of weight matrices)
Epoch 0: 5.277846891643054
Epoch 25: 0.002852904590187924
Epoch 50: 0.001472933405300805
Epoch 75: 0.0010099522693225351
Epoch 100: 0.0007743114402495745
Epoch 125: 0.0006305506805512219
Epoch 150: 0.000533297414676781
W1 size 85
W2 size 6
W3 size 3


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Using well-known algorithms: Logistic Regression, RandomForest and MLP


/usr/local/lib/python3.5/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Took: 28.087043046951294
--------------------------------------------------
En: 50	Tr: 84
Using first variation (Regressor and Classifier with score vectors)
epoch 0:	23.793800760363222
epoch 10:	23.3630744403808
epoch 20:	23.03267228930709
epoch 30:	22.83466582277227
epoch 40:	22.567400069012464
epoch 50:	22.288607910744986
Using second variation (average of outputs produced by each set of weight matrices)
Epoch 0: 1.3501209143972241
Epoch 25: 0.0031258158724611677
Epoch 50: 0.0014205130679607438
Epoch 75: 0.0009390675022213647
Epoch 100: 0.0007070440979166913
Epoch 125: 0.0005694254258697183
Epoch 150: 0.0004779539988304018
W1 size 106
W2 size 6
W3 size 3


...

Took: 230.0296061038971
--------------------------------------------------
En: 500	Tr: 422
Using first variation (Regressor and Classifier with score vectors)
epoch 0:	27.39200691512482
epoch 10:	27.16839772153587
epoch 20:	27.04829108057056
epoch 30:	26.894248068384304
epoch 40:	26.726854637126618
epoch 50:	26.55963785820566
Using second variation (average of outputs produced by each set of weight matrices)
Epoch 0: 0.047101801303673516
Epoch 25: 0.00019555911643070578
Epoch 50: 0.00010199585562988154
Epoch 75: 7.009595975100482e-05
Epoch 100: 5.391684224333517e-05
Epoch 125: 4.408453848255359e-05
Epoch 150: 3.744982887393341e-05
W1 size 487
W2 size 6
W3 size 3


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Using well-known algorithms: Logistic Regression, RandomForest and MLP


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Took: 231.95277643203735
--------------------------------------------------


In [ ]:
# HOW TO CHEAT LIKE A PRO
# """
# def test_selective(df_test, W1, W2, W3):
#     reset_graph()
#     x = tf.placeholder(tf.float32, [None, 300])
#     y = tf.placeholder(tf.float32, [None, 10]) # 1-10 => 10 classes

#     w1 = tf.placeholder(tf.float32, [300, 300])
#     w2 = tf.placeholder(tf.float32, [300, 300])
#     w3 = tf.placeholder(tf.float32, [300, 10])

#     b1 = tf.Variable(tf.zeros([300]))
#     b2 = tf.Variable(tf.zeros([300]))
#     b3 = tf.Variable(tf.zeros([10]))

#     l2 = tf.nn.sigmoid(tf.matmul(x, w1) + b1)
#     l3 = tf.nn.sigmoid(tf.matmul(l2, w2) + b2)
#     pred = tf.nn.softmax(tf.matmul(l3, w3) + b3)
    
#     correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
#     instance_accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#     with tf.Session() as sess:
#         sess.run(tf.global_variables_initializer())
    
#         # Testing the model
#         LSMR_test = preprocess_data(df_test)
#         X_test, y_test = get_test(LSMR_test)
#         accuracy = 0.
#         for i in range(len(X_test)):
#             best_instance_accuracy = float("-inf")
#             for language, score, movie_id in W1:
#                 w_1 = W1[(language, score, movie_id)]
#                 w_2 = W2[(language, score)]
#                 w_3 = W3[score]
#                 a = instance_accuracy.eval({x: np.atleast_2d(X_test[i]), y: np.atleast_2d(y_test[i]),
#                                    w1:w_1,
#                                    w2:w_2,
#                                    w3:w_3})
#                 if a > best_instance_accuracy:
#                     best_instance_accuracy = a
#             accuracy += best_instance_accuracy

#     return accuracy/len(X_test)
# """

# 3-layer NN > needs at least 3 days for training

In [ ]:
# gpu is a must
def train_deep(df_train, epochs=100, learning_rate=0.1, random_state=42):
    LSMR_train = preprocess_data(df_train)
    np.random.seed(random_state)
    data_dict, L1, L2, L3 = get_data_dict(LSMR_train, get_L2and3=True)
    init_weights = lambda layer, i, o: {k:2*np.random.random((i, o))-1 for k in layer}
    W1 = init_weights(L1, 300, 300)  # (languge, score, movie_id)
    W2 = init_weights(L2, 300, 300)  # (languge, score):
    W3 = init_weights(L3, 300, 10)  # score:
    
    
    reset_graph()
    x = tf.placeholder(tf.float32, [None, 300])
    y = tf.placeholder(tf.float32, [None, 10]) # 1-10 => 10 classes

    w1 = tf.Variable(tf.zeros([300, 300]))
    w2 = tf.Variable(tf.zeros([300, 300]))
    w3 = tf.Variable(tf.zeros([300, 10]))

    b1 = tf.Variable(tf.zeros([300]))
    b2 = tf.Variable(tf.zeros([300]))
    b3 = tf.Variable(tf.zeros([10]))

    l2 = tf.nn.sigmoid(tf.matmul(x, w1) + b1)
    l3 = tf.nn.sigmoid(tf.matmul(l2, w2) + b2)
    pred = tf.nn.softmax(tf.matmul(l3, w3) + b3)


    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
    training_curve = dict()
    with tf.device('/job:localhost/replica:0/task:0/device:GPU:0'):
        with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
            sess.run(tf.global_variables_initializer())
            for e in range(epochs+1):
                start = time.time()
                avg_cost = 0.
                for _, row in LSMR_train.iterrows():
                    score = row["Score"]
                    y_ = np.zeros(10)
                    y_[score-1] = 1
                    y_ = np.atleast_2d(y_)
                    x_ = np.atleast_2d(row["rev_vec"])
                    w_1, w_2, w_3 , _, c = sess.run([w1, w2, w3, optimizer, cost], feed_dict={x: x_,y: y_})               
                    avg_cost += c
                avg_cost /= len(LSMR_train)
                training_curve[e] = (avg_cost, time.time()-start)
                if e%10==0:
                    print("Epoch {}: {}".format(e, avg_cost))

    return w_1, w_2, w_3, training_curve

In [ ]:
def test_deep(df_test, w_1, w_2, w_3):
    reset_graph()
    x = tf.placeholder(tf.float32, [None, 300])
    y = tf.placeholder(tf.float32, [None, 10]) # 1-10 => 10 classes

    w1 = tf.placeholder(tf.float32, [300, 300])
    w2 = tf.placeholder(tf.float32, [300, 300])
    w3 = tf.placeholder(tf.float32, [300, 10])

    b1 = tf.Variable(tf.zeros([300]))
    b2 = tf.Variable(tf.zeros([300]))
    b3 = tf.Variable(tf.zeros([10]))

    l2 = tf.nn.sigmoid(tf.matmul(x, w1) + b1)
    l3 = tf.nn.sigmoid(tf.matmul(l2, w2) + b2)
    pred = tf.nn.softmax(tf.matmul(l3, w3) + b3)
    
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    with tf.device('/job:localhost/replica:0/task:0/device:GPU:0'):
        with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
            sess.run(tf.global_variables_initializer())

            # Testing the model
            LSMR_test = preprocess_data(df_test)
            X_test, y_test = get_test(LSMR_test)
            return accuracy.eval({x: X_test,
                                  y: y_test,
                                  w1:w_1,w2:w_2,
                                  w3:w_3})

In [ ]:
NUM_TRIALS = 1
scores_incremental = dict()
learning_curves = dict()
for i in range(NUM_TRIALS):
    scores_incremental[i] = dict()
    learning_curves[i] = dict()
    print("Trial:\t{}".format(i+1))
    k = 0
    skf = StratifiedKFold(n_splits=10, random_state=i)
    for train_index, test_index in skf.split(df["Review"], df["Language"]):
        start = time.time()
        w1, w2, w3, learning_curve = train_deep(df.loc[train_index], random_state=i, epochs=10000)
        s = test_deep(df.loc[test_index], w1, w2, w3)
        k += 1
        print("K:\t{}\nScore:\t{}".format(k, s))
        print("took:", time.time()-start)
        scores_incremental[i][k] = s
        learning_curves[i][k] = learning_curve
    print("*"*10)
    try:
        print("Trial {} avg score:\t {}".format(i+1, np.mean(list(scores_incremental[i].values()))))
    except:
        continue
    print("-"*30)